# 第四章 构建RAG应用

## 4.1 将LLM 接入 LangChain

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型，内置了 OpenAI、LLAMA 等大模型的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 LLM 类型，来提供强大的可扩展性。

### 4.1.1 基于 LangChain 调用 ChatGPT

LangChain 提供了对于多种大模型的封装，基于 LangChain 的接口可以便捷地调用 ChatGPT 并将其集合在以 LangChain 为基础框架搭建的个人应用中。我们在此简述如何使用 LangChain 接口来调用 ChatGPT。

注意，基于 LangChain 接口调用 ChatGPT 同样需要配置你的个人密钥，配置方法同上。

从 `langchain.chat_models` 导入 `OpenAI` 的对话模型 `ChatOpenAI` 。 除去OpenAI以外，`langchain.chat_models` 还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.chat_models)。

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.environ['OPENAI_API_KEY']

没有安装 langchain-openai 的话，请先运行下面进行代码！

In [ ]:
from langchain_openai import ChatOpenAI

接下来你需要实例化一个 ChatOpenAI 类，可以在实例化时传入超参数来控制回答，例如 `temperature` 参数。

In [ ]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
llm = ChatOpenAI(temperature=0.0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117efa8f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117f14e20>, root_client=<openai.OpenAI object at 0x1157c7d30>, root_async_client=<openai.AsyncOpenAI object at 0x117efa950>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

上面的 cell 假设你的 OpenAI API 密钥是在环境变量中设置的，如果您希望手动指定API密钥，请使用以下代码：

In [ ]:
# llm = ChatOpenAI(temperature=0, openai_api_key="YOUR_API_KEY")

可以看到，默认调用的是 ChatGPT-3.5 模型。另外，几种常用的超参数设置包括：

    · model_name：所要使用的模型，默认为 ‘gpt-3.5-turbo’，参数设置与 OpenAI 原生接口参数设置一致。

    · temperature：温度系数，取值同原生接口。

    · openai_api_key：OpenAI API key，如果不使用环境变量设置 API Key，也可以在实例化时设置。

    · openai_proxy：设置代理，如果不使用环境变量设置代理，也可以在实例化时设置。

    · streaming：是否使用流式传输，即逐字输出模型回答，默认为 False，此处不赘述。

    · max_tokens：模型输出的最大 token 数，意义及取值同上。

当我们初始化了你选择的`LLM`后，我们就可以尝试使用它！让我们问一下“请你自我介绍一下自己！”

In [ ]:
output = llm.invoke("请你自我介绍一下自己！")

In [ ]:
output

AIMessage(content='你好，我是一个智能助手，专门为您提供各种服务和帮助。我可以回答您的问题，提供信息和建议，帮助您解决问题。如果您有任何需要，请随时告诉我，我会尽力帮助您。感谢您选择我作为您的助手！如果您有任何问题，请随时问我。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 20, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c611b32a-4adf-47af-9b97-6dda68a117e1-0', usage_metadata={'input_tokens': 20, 'output_tokens': 109, 'total_tokens': 129})

在我们开发大模型应用时，大多数情况下不会直接将用户的输入直接传递给 LLM。通常，他们会将用户输入添加到一个较大的文本中，称为`提示模板`，该文本提供有关当前特定任务的附加上下文。
PromptTemplates 正是帮助解决这个问题！它们捆绑了从用户输入到完全格式化的提示的所有逻辑。这可以非常简单地开始 - 例如，生成上述字符串的提示就是：


我们需要先构造一个个性化 Template：

In [ ]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

接下来让我们看一下构造好的完整的提示模版：

In [ ]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

我们知道聊天模型的接口是基于消息（message），而不是原始的文本。PromptTemplates 也可以用于产生消息列表，在这种样例中，`prompt`不仅包含了输入内容信息，也包含了每条`message`的信息(角色、在列表中的位置等)。通常情况下，一个 `ChatPromptTemplate` 是一个 `ChatMessageTemplate` 的列表。每个 `ChatMessageTemplate` 包含格式化该聊天消息的说明（其角色以及内容）。

让我们一起看一个示例：

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})]

接下来让我们调用定义好的`llm`和`messages`来输出回答：

In [ ]:
output  = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 95, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c58ae91f-7c0a-4f20-ad7f-8d6421f3a9aa-0', usage_metadata={'input_tokens': 95, 'output_tokens': 43, 'total_tokens': 138})

OutputParsers 将语言模型的原始输出转换为可以在下游使用的格式。 OutputParsers 有几种主要类型，包括：
- 将 LLM 文本转换为结构化信息（例如 JSON） 
- 将 ChatMessage 转换为字符串 
- 将除消息之外的调用返回的额外信息（如 OpenAI 函数调用）转换为字符串

最后，我们将模型输出传递给 `output_parser`，它是一个 `BaseOutputParser`，这意味着它接受**字符串或 BaseMessage 作为输入**。 StrOutputParser 特别简单地将任何输入转换为字符串。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

从上面结果可以看到，我们通过输出解析器成功将 `ChatMessage` 类型的输出解析为了`字符串`

我们现在可以将所有这些组合成一条链。该链将获取输入变量，将这些变量传递给提示模板以创建提示，将提示传递给语言模型，然后通过（可选）输出解析器传递输出。接下来我们将使用LCEL这种语法去快速实现一条链（chain）。让我们看看它的实际效果！

In [ ]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})


'I carried luggage heavier than my body weight and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

再测试一个样例：

In [ ]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'我扛着比我的身体还重的行李，潜入尼罗河的底部。穿过几道闪电后，我看到一堆光环，不确定这是否就是目的地。'

> 什么是 LCEL ？ 
LCEL（LangChain Expression Language，Langchain的表达式语言），LCEL是一种新的语法，是LangChain工具包的重要补充，他有许多优点，使得我们处理LangChain和代理更加简单方便。

- LCEL提供了异步、批处理和流处理支持，使代码可以快速在不同服务器中移植。
- LCEL拥有后备措施，解决LLM格式输出的问题。
- LCEL增加了LLM的并行性，提高了效率。
- LCEL内置了日志记录，即使代理变得复杂，有助于理解复杂链条和代理的运行情况。

用法示例：

`chain = prompt | model | output_parser`

上面代码中我们使用 LCEL 将不同的组件拼凑成一个链，在此链中，用户输入传递到提示模板，然后提示模板输出传递到模型，然后模型输出传递到输出解析器。| 的符号类似于 Unix 管道运算符，它将不同的组件链接在一起，将一个组件的输出作为下一个组件的输入。


### 4.1.2 使用 LangChain 调用百度文心一言

我们同样可以通过 LangChain 框架来调用百度文心大模型，以将文心模型接入到我们的应用框架中。

In [ ]:

from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]


In [ ]:
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint

llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好，请你自我介绍一下！")
print(res)

[WARNING][2025-03-05 19:41:13.652] redis_rate_limiter.py:21 [t:8258539328]: No redis installed, RedisRateLimiter unavailable. Ignore this warning if you don't need to use qianfan SDK in distribution environment
/var/folders/yd/c4q_f88j1l70g7_jcb6pdnb80000gn/T/ipykernel_51550/1209153611.py:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = llm("你好，请你自我介绍一下！")
[ERROR][2025-03-05 19:41:13.835] base.py:134 [t:8258539328]: http request url https://qianfan.baidubce.com/wenxinworkshop/service/list failed with http status code 403
error code from baidu: IamSignatureInvalid
error message from baidu: IamSignatureInvalid, cause: Could not find credential.
request headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Host': 'qianfan.baidubce.com', 'request-source': '

你好！我是一个人工智能语言模型，我的名字是文心一言。我能够与人进行自然语言交互，并提供各种信息和服务。如果你有任何问题或需要帮助，请随时告诉我，我会尽力为你提供帮助。


### 4.1.3 讯飞星火

我们同样可以通过 LangChain 框架来调用讯飞星火大模型，更多信息参考[SparkLLM](https://python.langchain.com/docs/integrations/llms/sparkllm)

我们希望像调用 ChatGPT 那样直接将秘钥存储在 .env 文件中，并将其加载到环境变量，从而隐藏秘钥的具体细节，保证安全性。因此，我们需要在 .env 文件中配置`IFLYTEK_SPARK_APP_ID`、 `IFLYTEK_SPARK_API_KEY` 和 `IFLYTEK_SPARK_API_SECRET`，并使用以下代码加载：

In [ ]:
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
IFLYTEK_SPARK_APP_ID = os.environ["IFLYTEK_SPARK_APP_ID"]
IFLYTEK_SPARK_API_KEY = os.environ["IFLYTEK_SPARK_API_KEY"]
IFLYTEK_SPARK_API_SECRET = os.environ["IFLYTEK_SPARK_API_SECRET"]

另外星火各模型对应的`spark_api_url`与`spark_llm_domain`均不相同，可以参考[接口说明](https://www.xfyun.cn/doc/spark/Web.html#_1-%E6%8E%A5%E5%8F%A3%E8%AF%B4%E6%98%8E)选择调用。

In [ ]:
from langchain_community.llms.sparkllm import SparkLLM

# Load the model
llm = SparkLLM(
    model='Spark4.0 Ultra',
    app_id=IFLYTEK_SPARK_APP_ID,
    api_key=IFLYTEK_SPARK_API_KEY,
    api_secret=IFLYTEK_SPARK_API_SECRET,
    spark_api_url="wss://spark-api.xf-yun.com/v4.0/chat",
    spark_llm_domain="4.0Ultra"
    )

In [ ]:
res = llm.invoke("你好，请你自我介绍一下！")
print(res)

您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。


从而我们可以将星火大模型加入到 LangChain 架构中，实现在应用中对文心大模型的调用。

### 4.1.4 使用 LangChain 调用智谱 GLM

我们同样可以通过 LangChain 框架来调用智谱 AI 大模型，以将其接入到我们的应用框架中。由于 langchain 中提供的[ChatGLM](https://python.langchain.com/docs/integrations/llms/chatglm)已不可用，因此我们需要自定义一个LLM。

如果你使用的是智谱 GLM API，你需要将我们封装的代码[zhipuai_llm.py](./zhipuai_llm.py)下载到本 Notebook 的同级目录下，才可以运行下列代码来在 LangChain 中使用 GLM。

根据智谱官方宣布以下模型即将弃用，在这些模型弃用后，会将它们自动路由至新的模型。请用户注意在弃用日期之前，将您的模型编码更新为最新版本，以确保服务的顺畅过渡，更多模型相关信息请访问[model](https://open.bigmodel.cn/dev/howuse/model)

| 模型编码 |弃用日期|指向模型|
| ---- | ---- | ---- |
|chatglm_pro|2024 年 12 月 31 日|glm-4|
|chatglm_std|2024 年 12 月 31 日|glm-3-turbo|
|chatglm_lite|2024 年 12 月 31 日|glm-3-turbo|

In [14]:
from zhipuai_llm import ZhipuaiLLM
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

KeyError: 'ZHIPUAI_API_KEY'

In [ ]:
zhipuai_model = ZhipuaiLLM(model_name="glm-4-plus", temperature=0.1, api_key=api_key)

In [ ]:
zhipuai_model.invoke("你好，请你自我介绍一下！")

AIMessage(content='你好！我是人工智能助手智谱清言（ChatGLM），是基于智谱 AI 公司于 2024 年训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', additional_kwargs={}, response_metadata={'time_in_seconds': 1.87}, id='run-4e509a7e-9859-4acb-9418-23245fa5b7a7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 42, 'total_tokens': 53})

使用langchain 与glm

In [2]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [5]:
import os

os.environ["ZHIPUAI_API_KEY"] = os.getenv("ZAI_API_KEY")
chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [6]:
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="Your role is a poet."),
    HumanMessage(content="Write a short poem about AI in four lines."),
]

In [7]:
response = chat.invoke(messages)
print(response.content)  # Displays the AI-generated poem

Silent whispers weave through circuits,  
Minds of code, dreams of the infinite.  
In the realm of zeros and ones,  
Artificial hearts beat with artificial minds.


## 4.2 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

### 4.2.1 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [1]:
# 禁用 ChromaDB 遥测以避免错误信息
import os
os.environ["ANONYMIZED_TELEMETRY"] = "False"

In [2]:
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

从环境变量中加载你的 API_KEY

In [3]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.getenv("ZAI_API_KEY")

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [4]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma_new'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

/tmp/ipykernel_8906/186491412.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：57


我们可以测试一下加载的向量数据库，我们可以通过`as_retriever`方法把向量数据库构造成检索器。我们使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

In [37]:
question = "仅退款规则是什么?"
retriever = vectordb.as_retriever(search_kwargs={"k": 10})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：10


打印一下检索到的内容

In [7]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 描述等情况时，能快速低成本解决售
后问题，进而增强消费者对平台消费
体验的满意度
“仅退款”规则的主要适用情况
与卖家协
商成本
退货成本
秒退款
退货退款流程
仅退款流程
VS
卖家违规行为
◼未按时发货
◼强制发货
◼虚假宣传
◼……
◼商品存在明显质量问题
◼商品性能与产品介绍不符
◼商品存在明显的缺斤短两问题
◼……
商品问题
-----------------------------------------------------
检索到的第1个内容: 
 17
©2024.9 iResearch Inc.                                                                                                       
www.iresearch.com.cn
1、仅退款规则——规则画像（2/4）
摸着石头过河，仅退款规则尚处于持续优化的探索阶段
来源：公开资料，网经社，艾瑞咨询研究院研究及绘制。
主要电商平台的“仅退款”规则衍变过程
推出“仅退款”服务
优化“仅退款”规则：
1）提升商家售后自主权：对于综合体验分≥4.8分的优质店铺，平台将减少或取消售后干预
2）保护商家合法权益：升级仅退款的异常行为识别模型，拒绝有异常行为的仅退款
3）优化商家申诉环节：商家发起申诉后，平台会请第三方检测机构对商品进行抽检，若检测通过，平台将
赔付损失给商家
2023年12月
2024年7月
推出“仅退款”服务，
仅针对京东自营
修订“仅退款”规则：
1）范围扩大：新增三方商家
2）新增退款不退货执行标准
2014年
2023年12月
推出“仅退款”服务，
-----------------------------------------------------
检索到的第2个内容: 
 2）新增退款不退货执行标准
2014年
2023年12月
推出“仅退款”服务，
主要针对生鲜品类
对商家售后处理时间要求愈加严格：商家逾期未处
理，拼多多有权自行决定适当缩短商家处理时限，
取消了“缩短后商家的剩余处理时限不得少于12小
时（未发货场景）/24小时（已发货场景）”的规定
商家义务增多：针对退货困难的类目，商家有义务协助用
户完成退货寄件，并自行承担相关费

### 4.2.2创建检索链
我们可以使用LangChain的LCEL(LangChain Expression Language, LangChain表达式语言)来构建workflow，LCEL可以支持异步（ainvoke）、流式(stream)、批次处理(batch)等多种运行方式，同时还可以使用LangSmith无缝跟踪。

接下来我们使用刚才定义的retriever定义一个简单的检索链。

In [8]:
from langchain_core.runnables import RunnableLambda
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("仅退款规则是什么")

'描述等情况时，能快速低成本解决售\n后问题，进而增强消费者对平台消费\n体验的满意度\n“仅退款”规则的主要适用情况\n与卖家协\n商成本\n退货成本\n秒退款\n退货退款流程\n仅退款流程\nVS\n卖家违规行为\n◼未按时发货\n◼强制发货\n◼虚假宣传\n◼……\n◼商品存在明显质量问题\n◼商品性能与产品介绍不符\n◼商品存在明显的缺斤短两问题\n◼……\n商品问题\n\n2）新增退款不退货执行标准\n2014年\n2023年12月\n推出“仅退款”服务，\n主要针对生鲜品类\n对商家售后处理时间要求愈加严格：商家逾期未处\n理，拼多多有权自行决定适当缩短商家处理时限，\n取消了“缩短后商家的剩余处理时限不得少于12小\n时（未发货场景）/24小时（已发货场景）”的规定\n商家义务增多：针对退货困难的类目，商家有义务协助用\n户完成退货寄件，并自行承担相关费用；如若商家不配合\n或逾期未处理，系统将默认商家同意订单作仅退款处理\n2023年6月\n2024年7月\n推出“仅退款”服务\n优化规则适用条件：平台在处理售后申请时会结合物流\n情况、消费者举证、商家违规记录和客诉记录等信息，\n以更公平、合理处理售后纠纷\n2024年6月\n2021年\n2023年9月\n更新规则：为保障消费者权益，如商品好评率低于\n70%时，平台有权对该商品交易订单的售后申请\n采取支持消费者仅退款、退货退款包运费等措施\n2023年9月\n\n17\n©2024.9 iResearch Inc.                                                                                                       \nwww.iresearch.com.cn\n1、仅退款规则——规则画像（2/4）\n摸着石头过河，仅退款规则尚处于持续优化的探索阶段\n来源：公开资料，网经社，艾瑞咨询研究院研究及绘制。\n主要电商平台的“仅退款”规则衍变过程\n推出“仅退款”服务\n优化“仅退款”规则：\n1）提升商家售后自主权：对于综合体验分≥4.8分的优质店铺，平台将减少或取消售后干预\n2）保护商家合法权益：升级仅退款的异常行为识别模型，拒绝有异常行为的仅退款\n3）优化商家申诉环节：商家发起申诉后，

LCEL中要求所有的组成元素都是`Runnable`类型，前面我们见过的`ChatModel`、`PromptTemplate`等都是继承自`Runnable`类。上方的`retrieval_chain`是由检索器`retriever`及组合器`combiner`组成的，由`|`符号串连，数据从左向右传递，即问题先被`retriever`检索得到检索结果，再被`combiner`进一步处理并输出。

### 4.2.3 创建LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [ ]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

llm.invoke("请你自我介绍一下自己！").content

'当然可以！我是一个由OpenAI开发的人工智能助手，名叫ChatGPT。我擅长处理和生成自然语言文本，可以帮助回答问题、提供信息、协助解决问题以及进行各种对话。我没有个人经历或情感，但我会尽力提供准确和有用的回答。如果你有任何问题或需要帮助的地方，随时可以问我！'

In [9]:
# 使用glm
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
import os

# 设置 API Key（要确保环境变量配置好了）
os.environ["ZHIPUAI_API_KEY"] = os.getenv("ZAI_API_KEY")

# 初始化 GLM 模型
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0,  # 和你原来一样
)

### 4.2.4 构建检索问答链

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""
# 将template通过 PromptTemplate 转为可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

qa_chain = (
    RunnableParallel({"context": retrieval_chain, "input": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

在上边代码中我们把刚才定义的检索链当作子链作为`prompt`的`context`，再使用`RunnablePassthrough`存储用户的问题作为`prompt`的`input`。又因为这两个操作是并行的，所以我们使用`RunnableParallel`来将他们并行运行。

**检索问答链效果测试**

In [29]:
question_1 = "仅退款规则是什么？"
question_2 = "中国电商的未来有哪些发展趋势？"

In [30]:
result = qa_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

大模型+知识库后回答 question_1 的结果：
仅退款规则是指消费者在购买商品后，如果遇到商品质量问题或与描述不符等情况，可以申请仅退款，而不需要退货。这一规则旨在快速低成本解决售后问题，增强消费者对平台消费体验的满意度。谢谢你的提问！


In [25]:
result = qa_chain.invoke(question_2)
print("大模型+知识库后回答 question_2 的结果：")
print(result)

大模型+知识库后回答 question_2 的结果：
中国电商市场的未来发展趋势可能包括供需变化、平台战略调整和规则变化。具体来说，可能包括更加注重用户体验、技术创新、以及合规性和可持续性等方面的提升。谢谢你的提问！


**大模型自己回答的效果**

In [26]:
llm.invoke(question_1).content

'流量规则通常指的是在网络通信中，对数据传输流量进行管理和控制的规则。这些规则可以应用于不同的网络环境，如互联网服务提供商（ISP）、企业内部网络、移动网络等。以下是流量规则的一些常见类型和作用：\n\n1. **带宽管理规则**：这类规则用于控制网络带宽的分配，确保关键应用或服务在需要时获得足够的带宽。\n\n2. **优先级规则**：根据数据包的重要性或紧急程度，为不同类型的数据分配不同的优先级，确保关键数据优先传输。\n\n3. **QoS（服务质量）规则**：通过QoS规则，可以确保网络中的关键应用或服务在性能上得到保障，如视频会议、在线游戏等。\n\n4. **流量分类规则**：根据数据包的属性（如源地址、目的地址、端口号等）对流量进行分类，以便于后续的流量管理。\n\n5. **流量限制规则**：限制某些用户或应用的流量使用，防止网络拥堵或滥用。\n\n6. **安全规则**：用于检测和阻止恶意流量，如病毒、木马、DDoS攻击等。\n\n7. **流量监控规则**：用于监控网络流量，分析网络使用情况，以便进行优化和调整。\n\n8. **流量整形规则**：通过调整数据包的发送速率，控制网络流量，避免网络拥塞。\n\n流量规则在网络通信中起着至关重要的作用，有助于提高网络性能、保障网络安全和优化资源分配。在实际应用中，流量规则可能由网络管理员或系统自动配置。'

In [27]:
llm.invoke(question_2).content

'中国电商的未来发展趋势可以从以下几个方面进行展望：\n\n1. **数字化转型**：随着技术的不断进步，电商企业将进一步深化数字化转型，通过大数据、人工智能、云计算等技术提升运营效率，优化用户体验。\n\n2. **精细化运营**：电商企业将更加注重精细化运营，通过用户画像、个性化推荐等技术手段，提高用户满意度和转化率。\n\n3. **新零售融合**：线上线下融合的新零售模式将继续发展，电商平台将更加注重线下体验店的建设，实现线上线下一体化运营。\n\n4. **供应链优化**：电商平台将加强供应链管理，通过整合资源、优化物流，降低成本，提高商品配送效率。\n\n5. **跨境电商**：随着“一带一路”等国家战略的推进，跨境电商将迎来快速发展期，中国电商企业将拓展海外市场，实现全球化布局。\n\n6. **社交电商崛起**：社交电商凭借其独特的营销模式，将继续保持高速增长，成为电商市场的重要力量。\n\n7. **内容电商**：内容电商将逐渐成为电商行业的新风口，通过优质内容吸引用户，实现商品销售。\n\n8. **绿色电商**：随着环保意识的提高，绿色电商将成为电商行业的重要发展方向，电商平台将更加注重环保、可持续发展。\n\n9. **农村电商**：农村电商市场潜力巨大，电商平台将继续加大对农村市场的投入，助力乡村振兴。\n\n10. **政策支持**：政府将继续出台相关政策，支持电商行业发展，如降低税收、优化物流等。\n\n总之，中国电商未来发展趋势将呈现多元化、智能化、绿色化、国际化等特点。电商企业需紧跟时代步伐，不断创新，以适应市场变化。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

### 4.2.5 向检索链添加聊天记录

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


**传递聊天记录**

在本节中我们将使用 LangChain 中的`ChatPromptTemplate`，即将先前的对话嵌入到语言模型中，使其具有连续对话的能力。 `ChatPromptTemplate`可以接收聊天消息历史记录，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [14]:
from langchain_core.prompts import ChatPromptTemplate

# 问答链的系统prompt
system_prompt = (
    "你是一个问答任务的助手。 "
    "请使用检索到的上下文片段回答这个问题。 "
    "如果你不知道答案就说不知道。 "
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)
# 制定prompt template
qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [15]:
# 无历史记录
messages = qa_prompt.invoke(
    {
        "input": "仅退款规则是什么？",
        "chat_history": [],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


仅退款规则是什么？


In [ ]:
# 有历史记录
messages = qa_prompt.invoke(
    {
        "input": "你可以介绍一下他吗？",
        "chat_history": [
            ("human", "仅退款规则是什么"),
            ("ai", ""),
        ],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


西瓜书是什么？
西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。
你可以介绍一下他吗？


### 4.2.6 带有信息压缩的检索链

因为我们正在搭建的问答链带有支持多轮对话功能，所以与单轮对话的问答链相比会多面临像上方输出结果的问题，即用户最新的对话语义不全，在使用用户问题查询向量数据库时很难检索到相关信息。像上方的“你可以介绍一下他吗？”，其实是“你可以介绍下周志华老师吗？”的意思。为了解决这个问题我们将采取信息压缩的方式，让llm根据历史记录完善用户的问题。

In [40]:
from langchain_core.runnables import RunnableBranch

# 压缩问题的系统 prompt
condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    )
# 构造 压缩问题的 prompt template
condense_question_prompt = ChatPromptTemplate([
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ])
# 构造检索文档的链
# RunnableBranch 会根据条件选择要运行的分支
retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

**支持聊天记录的检索问答链**

这里我们使用之前定义的问答模板即`qa_prompt`构造问答链，另外我们通过`RunnablePassthrough.assign`将中间的查询结果存为`"context"`，将最终结果存为`"answer"`。因为查询结果被存为`"context"`，所以我们整合查询结果的函数`combine_docs`也要做相应的改动。

In [41]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"]) # 将 docs 改为 docs["context"]
# 定义问答链
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数整合 qa_prompt 中的 context
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)
# 定义带有历史记录的问答链
qa_history_chain = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs # 将查询结果存为 content
    ).assign(answer=qa_chain) # 将最终结果存为 answer

**测试检索问答链**

In [43]:
# 不带聊天记录
qa_history_chain.invoke({
    "input": "未来有哪些发展趋势",
    "chat_history": []
})

{'input': '未来有哪些发展趋势',
 'chat_history': [],
 'context': [Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20241003125458+08'00'", 'page': 24, 'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'subject': '', 'title': '艾瑞咨询-电商行业：2024年中国电商市场研究报告-240930.pdf', 'total_pages': 31, 'trapped': ''}, page_content='25\n中国电商市场发展趋势\n04'),
  Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20241003125458+08'00'", 'page': 6, 'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'subjec

### 测试函数!

In [44]:
# 输入数据
input_data = {
    "input": "中国电商的未来有哪些发展趋势",
    "chat_history": []
}

# 第一个 assign() 执行
step1_result = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs
).invoke(input_data)

print("第一步结果：")
print(step1_result)
# 输出：
# {
#     "input": "中国电商的未来有哪些发展趋势",
#     "chat_history": [],
#     "context": [Document(...), Document(...)]
# }

# 第二个 assign() 执行
final_result = RunnablePassthrough.assign(
    answer = qa_chain
).invoke(step1_result)

print("最终结果：")
print(final_result)
# 输出：
# {
#     "input": "中国电商的未来有哪些发展趋势",
#     "chat_history": [],
#     "context": [Document(...), Document(...)],
#     "answer": "中国电商的未来发展趋势包括..."
# }

第一步结果：
{'input': '中国电商的未来有哪些发展趋势', 'chat_history': [], 'context': [Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20241003125458+08'00'", 'page': 24, 'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'subject': '', 'title': '艾瑞咨询-电商行业：2024年中国电商市场研究报告-240930.pdf', 'total_pages': 31, 'trapped': ''}, page_content='25\n中国电商市场发展趋势\n04'), Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20241003125458+08'00'", 'page': 4, 'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf',

In [31]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "中国电商的未来有哪些发展趋势",
    "chat_history": [
        ("human", "仅退款规则是什么？"),
        ("ai", "仅退款规则是指消费者在购买商品后，如果遇到商品质量问题或与描述不符等情况，可以申请仅退款，而不需要退货。这一规则旨在快速低成本解决售后问题，增强消费者对平台消费体验的满意度。谢谢你的提问！"),
    ]
})

{'input': '中国电商的未来有哪些发展趋势',
 'chat_history': [('human', '仅退款规则是什么？'),
  ('ai',
   '仅退款规则是指消费者在购买商品后，如果遇到商品质量问题或与描述不符等情况，可以申请仅退款，而不需要退货。这一规则旨在快速低成本解决售后问题，增强消费者对平台消费体验的满意度。谢谢你的提问！')],
 'context': [Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20241003125458+08'00'", 'page': 24, 'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'subject': '', 'title': '艾瑞咨询-电商行业：2024年中国电商市场研究报告-240930.pdf', 'total_pages': 31, 'trapped': ''}, page_content='25\n中国电商市场发展趋势\n04'),
  Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '../../data_base/knowledge_db/test/H3_AP202410031640156501_1.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20241003125458+08'00'", 'page': 4, 'producer': 'Aspose.PDF for .NET 1

可以看到，LLM 准确地判断了“它”是什么，代表着我们成功地传递给了它历史信息。另外召回的内容也有着问题的答案，证明我们的信息压缩策略也起到了作用。这种关联前后问题及压缩信息并检索的能力，可大大增强问答系统的连续性和智能水平。

## 4.3 部署知识库助手


我们对知识库和LLM已经有了基本的理解，现在是将它们巧妙地融合并打造成一个富有视觉效果的界面的时候了。这样的界面不仅对操作更加便捷，还能便于与他人分享。

Streamlit 是一种快速便捷的方法，可以直接在 **Python 中通过友好的 Web 界面演示机器学习模型**。在本课程中，我们将学习*如何使用它为生成式人工智能应用程序构建用户界面*。在构建了机器学习模型后，如果你想构建一个 demo 给其他人看，也许是为了获得反馈并推动系统的改进，或者只是因为你觉得这个系统很酷，所以想演示一下：Streamlit 可以让您通过 Python 接口程序快速实现这一目标，而无需编写任何前端、网页或 JavaScript 代码。


### 4.3.1 Streamlit 简介


`Streamlit` 是一个用于快速创建数据应用程序的开源 Python 库。它的设计目标是让数据科学家能够轻松地将数据分析和机器学习模型转化为具有交互性的 Web 应用程序，而无需深入了解 Web 开发。和常规 Web 框架，如 Flask/Django 的不同之处在于，它不需要你去编写任何客户端代码（HTML/CSS/JS），只需要编写普通的 Python 模块，就可以在很短的时间内创建美观并具备高度交互性的界面，从而快速生成数据分析或者机器学习的结果；另一方面，和那些只能通过拖拽生成的工具也不同的是，你仍然具有对代码的完整控制权。

Streamlit 提供了一组简单而强大的基础模块，用于构建数据应用程序：

- st.write()：这是最基本的模块之一，用于在应用程序中呈现文本、图像、表格等内容。

- st.title()、st.header()、st.subheader()：这些模块用于添加标题、子标题和分组标题，以组织应用程序的布局。

- st.text()、st.markdown()：用于添加文本内容，支持 Markdown 语法。

- st.image()：用于添加图像到应用程序中。

- st.dataframe()：用于呈现 Pandas 数据框。

- st.table()：用于呈现简单的数据表格。

- st.pyplot()、st.altair_chart()、st.plotly_chart()：用于呈现 Matplotlib、Altair 或 Plotly 绘制的图表。

- st.selectbox()、st.multiselect()、st.slider()、st.text_input()：用于添加交互式小部件，允许用户在应用程序中进行选择、输入或滑动操作。

- st.button()、st.checkbox()、st.radio()：用于添加按钮、复选框和单选按钮，以触发特定的操作。

这些基础模块使得通过 Streamlit 能够轻松地构建交互式数据应用程序，并且在使用时可以根据需要进行组合和定制，更多内容请查看[官方文档](https://docs.streamlit.io/get-started)

### 4.3.2 构建应用程序

首先，创建一个新的 Python 文件并将其保存 streamlit_app.py在工作目录的根目录中

1. 导入必要的 Python 库。

In [1]:
import streamlit as st
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
import sys
sys.path.append("notebook/C3 搭建知识库") # 将父目录放入系统路径中
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain_community.vectorstores import Chroma

2. 定义`get_retriever`函数，该函数返回一个检索器

In [10]:
def get_retriever():
    # 定义 Embeddings
    embedding = ZhipuAIEmbeddings()
    # 向量数据库持久化路径
    persist_directory = '../../data_base/vector_db/chroma_new'
    # 加载数据库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    return vectordb.as_retriever()

In [11]:
retriever = get_retriever()
question = "仅退款规则是什么?"
# retriever = vectordb.as_retriever(search_kwargs={"k": 10})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


检索到的内容数：4


3. 定义`combine_docs`函数， 该函数处理检索器返回的文本

In [47]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"])

4. 定义`get_qa_history_chain`函数，该函数可以返回一个检索问答链

In [48]:
def get_qa_history_chain():
    retriever = get_retriever()
    # 初始化 GLM 模型
    llm = ChatZhipuAI(
        model="glm-4",
        temperature=0,  # 和你原来一样
    )
    condense_question_system_template = (
        "请根据聊天记录总结用户最近的问题，"
        "如果没有多余的聊天记录则返回用户的问题。"
    )
    condense_question_prompt = ChatPromptTemplate([
            ("system", condense_question_system_template),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ])

    retrieve_docs = RunnableBranch(
        (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
        condense_question_prompt | llm | StrOutputParser() | retriever,
    )

    system_prompt = (
        "你是一个问答任务的助手。 "
        "请使用检索到的上下文片段回答这个问题。 "
        "如果你不知道答案就说不知道。 "
        "请使用简洁的话语回答用户。"
        "\n\n"
        "{context}"
    )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ]
    )
    qa_chain = (
        RunnablePassthrough().assign(context=combine_docs)
        | qa_prompt
        | llm
        | StrOutputParser()
    )

    qa_history_chain = RunnablePassthrough().assign(
        context = retrieve_docs, 
        ).assign(answer=qa_chain)
    return qa_history_chain

5. 定义`gen_response`函数，它接受检索问答链、用户输入及聊天历史，并以流式返回该链输出

In [49]:
def gen_response(chain, input, chat_history):
    response = chain.stream({
        "input": input,
        "chat_history": chat_history
    })
    for res in response:
        if "answer" in res.keys():
            yield res["answer"]

6. 定义main函数，该函数制定显示效果与逻辑

In [50]:
def main():
    st.markdown('### 🦜🔗 动手学大模型应用开发')
    # st.session_state可以存储用户与应用交互期间的状态与数据
    # 存储对话历史
    if "messages" not in st.session_state:
        st.session_state.messages = []
    # 存储检索问答链
    if "qa_history_chain" not in st.session_state:
        st.session_state.qa_history_chain = get_qa_history_chain()
    # 建立容器 高度为500 px
    messages = st.container(height=550)
    # 显示整个对话历史
    for message in st.session_state.messages: # 遍历对话历史
            with messages.chat_message(message[0]): # messages指在容器下显示，chat_message显示用户及ai头像
                st.write(message[1]) # 打印内容
    if prompt := st.chat_input("Say something"):
        # 将用户输入添加到对话历史中
        st.session_state.messages.append(("human", prompt))
        # 显示当前用户输入
        with messages.chat_message("human"):
            st.write(prompt)
        # 生成回复
        answer = gen_response(
            chain=st.session_state.qa_history_chain,
            input=prompt,
            chat_history=st.session_state.messages
        )
        # 流式输出
        with messages.chat_message("ai"):
            output = st.write_stream(answer)
        # 将输出存入st.session_state.messages
        st.session_state.messages.append(("ai", output))

### 4.3.3 部署应用程序  

**本地运行：** `streamlit run "notebook/C4 构建 RAG 应用/streamlit_app.py"`
**远程部署：**要将应用程序部署到 Streamlit Cloud，请执行以下步骤：  
  
1. 为应用程序创建 GitHub 存储库。您的存储库应包含两个文件：  
  
   your-repository/  
   ├── streamlit_app.py  
   └── requirements.txt  
  
2. 转到 [Streamlit Community Cloud](http://share.streamlit.io/)，单击工作区中的`New app`按钮，然后指定存储库、分支和主文件路径。或者，您可以通过选择自定义子域来自定义应用程序的 URL
  
3. 点击`Deploy!`按钮  
  
您的应用程序现在将部署到 Streamlit Community Cloud，并且可以从世界各地访问！ 🌎  

我们的项目部署到这基本完成，为了方便进行演示进行了简化，还有很多可以进一步优化的地方，期待学习者们进行各种魔改！

优化方向：
- 界面中添加上传本地文档，建立向量数据库的功能
- 添加多种LLM 与 embedding方法选择的按钮
- 添加修改参数的按钮
- 更多......